# Intro

As a first test for DKT elements, in this notebook we find minimal configurations (of the vertical displacements $u$) for the energy

$$ I_{K i}(w) = \frac{1}{2} \int_\omega |D^2 w|^2 \mathrm{d}x - \int_\omega f_3 w \mathrm{d}x$$

under homogeneous Dirichlet boundary conditions. This means solving the linear problem

$$ (D^2w, D^2v) = (f_3,v),$$

for all $v \in W$, with the discretisation

$$ (\nabla \nabla_h w_h, \nabla \nabla_h v_h) = (f_3, v_h), $$

for all $v_h \in W_{h,D} := ...$

The crux of the matter is the [discrete gradient operator](discrete_gradient.ipynb) $\nabla_h$. Recall that $\nabla_h : W_h \rightarrow \Theta_h$ is defined as...

Computing the gradient on whole Functions is not what we need here, though. Instead we need to incorporate it cell-wise into the assembly process. This requires writing our own `ufc::form` and `ufc::cell_integral`, etc. for dolfin's assembler to use.

## Failing test

In [ ]:
from dolfin import *
import nbimporter
from discrete_gradient import dkt_gradient_operator, compute_dkt_gradient

W = FunctionSpace(UnitSquareMesh(1,1), 'DKT', 3)
T = VectorFunctionSpace(W.mesh(), 'Lagrange', 2, dim=2)
w = TrialFunction(W)
v = TestFunction(W)

Dh = dkt_gradient_operator(W, T)

class Boundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

bc = DirichletBC(W, Constant(0.), Boundary())

def gradh(u):
    # FIXME: won't work for Arguments!
    return compute_dkt_gradient(W, T, u)

a = inner(grad(gradh(w)), grad(gradh(v)))*dx
L = inner(Constant(-9.8), v)*dx

u = Function(W)
solve(a == L, u)

## ufc_utils

In [ ]:
from ffc.backends.ufc import form as uf

In [ ]:
impl = dict(classname = 'linear_kirchhoff',
            constructor = '// Do nothing',
            constructor_arguments = '',
            create_cell_integral = '',
            create_coordinate_dofmap = '',
            create_coordinate_finite_element = '',
            create_coordinate_mapping = '',
            create_custom_integral = '',
            create_cutcell_integral = '',
            create_default_cell_integral = '',
            create_default_custom_integral = '',
            create_default_cutcell_integral = '',
            create_default_exterior_facet_integral = '',
            create_default_interface_integral = '',
            create_default_interior_facet_integral = '',
            create_default_overlap_integral = '',
            create_default_vertex_integral = '',
            create_dofmap = '',
            create_exterior_facet_integral = '',
            create_finite_element = '',
            create_interface_integral = 'return 0;',
            create_interior_facet_integral = 'return 0;',
            create_overlap_integral = 'return 0;',
            create_vertex_integral = '',
            destructor = '// Do nothing',
            has_cell_integrals = '',
            has_custom_integrals = '',
            has_cutcell_integrals = '',
            has_exterior_facet_integrals = '',
            has_interface_integrals = '',
            has_interior_facet_integrals = '',
            has_overlap_integrals = '',
            has_vertex_integrals = '',
            initializer_list = '',
            max_cell_subdomain_id = '',
            max_custom_subdomain_id = '',
            max_cutcell_subdomain_id = '',
            max_exterior_facet_subdomain_id = '',
            max_interface_subdomain_id = '',
            max_interior_facet_subdomain_id = '',
            max_overlap_subdomain_id = '',
            max_vertex_subdomain_id = '',
            members = '',
            num_coefficients = '',
            original_coefficient_position = '',
            rank = '',
            signature = 'return "Linear Kirchhoff plate";')
print(uf.form_header % impl)
print(uf.form_implementation % impl)

In [ ]:
implementation = {}

implementation["signature"] = 'return "my form"'
implementation["rank"] = "return 2;" 
implementation["num_coefficients"] = "return 0;" 
implementation["num_cell_domains"] = "return 3;"
implementation["num_interior_facet_domains"] = "return 1;" 
implementation["num_exterior_facet_domains"] = "return 0;" 

implementation["create_finite_element"] = """
switch (i)
{
case 0:
  return new my_finite_element_0();
case 1:
  return new my_finite_element_1();
default:
return 0; }"""
implementation["create_dofmap"] = """ switch (i)
{
case 0:
  return new my_dofmap_0();
case 1:
  return new my_dofmap_1();
default:
return 0; }"""
implementation["create_cell_integral"] = """ switch (i)
{
case 0:
  return new my_cell_integral_0();
case 1:
  return new my_cell_integral_1();
case 2:
  return new my_cell_integral_2();
default:
return 0; }"""
implementation["create_exterior_facet_integral"] = "return new my_exterior_facet_integral();"